In [199]:
import pandas as pd
import numpy as np

from sklearn.feature_extraction.text import CountVectorizer

In [200]:
df = pd.read_csv("PetaBencana_Dataset - PetaBencana_Dataset.csv")

df.head()

,pkey,date,time,source,status,url,image_url,disaster_type,report_dat,tags,...,ID_Desa,ID_Kec,ID_Kab,ID_Prov,Desa,Kecamatan,Kabupaten,Provinsi,long,lat
0,94941,01/01/2020,00:03:02,grasp,confirmed,2042596d-b551-4ef2-803d-1ee964cf2278,https://images.petabencana.id/2042596d-b551-4e...,flood,"{ ""report_type"": ""flood"", ""flood_depth"": 30 }","{ ""district_id"": ""3174"", ""local_area_id"": ""125...",...,3.174020e+09,3174020.0,3174.0,31.0,Duri Kepa,Kebon Jeruk,Jakarta Barat,Daerah Khusus Ibukota Jakarta,106.76866,-6.16978
1,95061,01/01/2020,00:43:02,grasp,confirmed,5a0ad45f-7020-43b7-882f-672487ebc6d8,https://images.petabencana.id/5a0ad45f-7020-43...,flood,"{ ""points"": 1, ""flood_depth"": 80, ""report_type...","{ ""district_id"": null, ""local_area_id"": null, ...",...,3.275061e+09,3275061.0,3275.0,32.0,Pejuang,Medan Satria,Bekasi,Jawa Barat,106.97870,-6.17944
2,95062,01/01/2020,01:14:53,grasp,confirmed,e065b40d-2084-48d6-b42f-1f98cb33e4cc,NaN,flood,"{ ""points"": 1, ""flood_depth"": 51, ""report_type...","{ ""district_id"": ""3172"", ""local_area_id"": ""178...",...,3.172060e+09,3172060.0,3172.0,31.0,Cipinang Muara,Jatinegara,Jakarta Timur,Daerah Khusus Ibukota Jakarta,106.89100,-6.23449
3,95063,01/01/2020,01:15:08,grasp,confirmed,7cb26903-b1c8-4be7-8b81-7014c8a83fbd,https://images.petabencana.id/7cb26903-b1c8-4b...,flood,"{ ""points"": 2, ""flood_depth"": 100, ""report_typ...","{ ""district_id"": null, ""local_area_id"": null, ...",...,3.216061e+09,3216061.0,3216.0,32.0,Mekarmukti,Cikarang Utara,Bekasi,Jawa Barat,107.16189,-6.30170
4,95064,01/01/2020,01:18:03,grasp,confirmed,570d2cb8-d5b6-404f-86ef-8a2a0d27f391,https://images.petabencana.id/570d2cb8-d5b6-40...,flood,"{ ""points"": 1, ""flood_depth"": 15, ""report_type...","{ ""district_id"": ""3173"", ""local_area_id"": ""193...",...,3.173060e+09,3173060.0,3173.0,31.0,Cempaka Baru,Kemayoran,Jakarta Pusat,Daerah Khusus Ibukota Jakarta,106.86092,-6.16859


In [201]:
print("Bencana alam yang dilaporkan: ")
print(df["disaster_type"].unique())

Bencana alam yang dilaporkan: 
['flood' 'earthquake' 'wind' 'fire' 'volcano' 'haze']


In [202]:
# Akan dibuat model NLP, sehingga baris-baris di mana kolom "text" kosong tidak akan berguna
df = df.dropna(subset=["text"])

In [203]:
# buat dataframe berdasarkan tipe bencana

flood = df[df["disaster_type"] == "flood"]
earthquake = df[df["disaster_type"] == "earthquake"]
wind = df[df["disaster_type"] == "wind"]
fire = df[df["disaster_type"] == "fire"]
volcano = df[df["disaster_type"] == "volcano"]
haze = df[df["disaster_type"] == "haze"]

In [204]:
print(f"flood DataFrame memiliki {flood.shape[0]} observasi")
print(f"earthquake DataFrame memiliki {earthquake.shape[0]} observasi")
print(f"wind DataFrame memiliki {wind.shape[0]} observasi")
print(f"fire DataFrame memiliki {fire.shape[0]} observasi")
print(f"volcano DataFrame memiliki {volcano.shape[0]} observasi")
print(f"haze DataFrame memiliki {haze.shape[0]} observasi")

flood DataFrame memiliki 3630 observasi
earthquake DataFrame memiliki 112 observasi
wind DataFrame memiliki 57 observasi
fire DataFrame memiliki 12 observasi
volcano DataFrame memiliki 17 observasi
haze DataFrame memiliki 2 observasi


In [205]:
flood_sample = flood[["disaster_type", "text"]].sample(100, random_state=123)

earthquake_sample = earthquake[["disaster_type", "text"]].sample(100, random_state=123)

# 
train_set = pd.concat([flood_sample[:70], earthquake_sample[:70]])
train_x = train_set["text"]
train_y = train_set["disaster_type"]


test_set = pd.concat([flood_sample[70:], earthquake_sample[70:]])
test_x = test_set["text"]
test_y = test_set["disaster_type"]

# Bag Of Words #1

In [206]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn import svm

In [207]:
vectorizer = CountVectorizer()
train_x_vectors = vectorizer.fit_transform(train_x)

In [211]:
# classifier support vector machine
clf_svm = svm.SVC()
clf_svm.fit(train_x_vectors, train_y)

SVC()

In [212]:
test_x_vectors = vectorizer.transform(test_x)

print(f"Test set score: {np.mean([clf_svm.predict(test_x_vectors) == np.array(test_y)])}")

Test set score: 0.8166666666666667


# Bag Of Words #2
this time with some settings

In [213]:
vectorizer = CountVectorizer(binary=True)  # n_gram , 2 word phrases. binary?
train_x_vectors = vectorizer.fit_transform(train_x)

In [214]:
# classifier support vector machine
clf_svm = svm.SVC(kernel="linear")  # kernel = "linear" good for text
clf_svm.fit(train_x_vectors, train_y)

SVC(kernel='linear')

In [215]:
test_x_vectors = vectorizer.transform(test_x)

print(f"Test set score: {np.mean([clf_svm.predict(test_x_vectors) == np.array(test_y)])}")

Test set score: 0.8833333333333333


# Word Vectors

In [2]:
# install spacy (if haven't already)

# !pip install spacy
# !python -m spacy download en_core_web_md

In [216]:
import spacy

In [217]:
nlp = spacy.load("en_core_web_md")

In [218]:
train_docs = [nlp(text) for text in train_x]
train_x_word_vectors = [x.vector for x in train_docs]

In [219]:
clf_svm_wv = svm.SVC(kernel="linear")  # linear good for text
clf_svm_wv.fit(train_x_word_vectors, train_y)

SVC(kernel='linear')

In [220]:
test_docs = [nlp(text) for text in test_x]  # nlp(text), averages all words embedding 
test_x_word_vectors = [x.vector for x in test_docs]

print(f"Test set score: {np.mean([clf_svm_wv.predict(test_x_word_vectors) == np.array(test_y)])}")

Test set score: 0.5166666666666667


# Stopwords removal
- removing unnecessary words to improve accuracy
- https://www.geeksforgeeks.org/removing-stop-words-nltk-python/

# Bag of Words #3 (with stopwords removal)

In [221]:
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

stop_words = stopwords.words("indonesian")

train_x_sr = []
for phrase in train_x:
    
    words = word_tokenize(phrase)

    stripped_phrase = []


    for word in word_tokenize(phrase):
        if word not in stop_words:
            stripped_phrase.append(word)

    stripped_phrase
    phrase = " ".join(stripped_phrase)
    train_x_sr.append(phrase)

# train_x_sr

In [222]:
vectorizer = CountVectorizer()
train_x_vectors = vectorizer.fit_transform(train_x_sr)

In [223]:
# classifier support vector machine
clf_svm = svm.SVC(kernel="linear")  # kernel = "linear" good for text
clf_svm.fit(train_x_vectors, train_y)

SVC(kernel='linear')

In [224]:
test_x_sr = []
for phrase in test_x:
    
    words = word_tokenize(phrase)

    stripped_phrase = []


    for word in word_tokenize(phrase):
        if word not in stop_words:
            stripped_phrase.append(word)

    stripped_phrase
    phrase = " ".join(stripped_phrase)
    test_x_sr.append(phrase)

In [225]:
test_x_vectors = vectorizer.transform(test_x_sr)

print(f"Test set score: {np.mean([clf_svm.predict(test_x_vectors) == np.array(test_y)])}")

Test set score: 0.8666666666666667
